In [1]:
import random
import math
import matplotlib.pyplot as plt
import time
import numpy as np
from point import Point
import json

# Load the Layout Pool from Static Generation

In [2]:
with open("data/static_layout_pool.json", 'r') as openfile:
    json_object = json.load(openfile)

layouts = [[Point(p[0], p[1]) for p in lay] for lay in json_object]

# Dynamic Trial Shuffling
- We shuffle the N trials such that the boundary visibility alternates each time.
- We read N/2 trials from static generation.
- Boundary visibility is alternating, so we simply assume border_visibility = i % 2

## Generation

- At least K = 5 trials between trials of the same pairing.
- We make sure that between adjacent pairs, the ending location and starting location is always opposite.
    - Also, make sure these distances obey MIN_DIST_PATH

In [22]:
NUM_SHUFFLES = 50
DIST_BETWEEN_COPY = 5 # K
LAYOUT_COUNT = len(layouts)

In [25]:
def generate_random_shuffle():
    # we interleave to random lists of 1...n/2, therefore it is guaranteed that the random shuffling
    #is valid for the constraint: boundary visibility alternates each time. each layout gets exactly 
    #one case with border, exactly one without border
    def random_half_shuffle():
        l = list(range(len(layouts)))
        random.shuffle(l)
        return l
    
    l = [x for t in zip(random_half_shuffle(), random_half_shuffle()) for x in t]
    return l
    


def is_valid_shuffle(order):
    # At least K = 5 trials between trials of the same pairing.
    # since we generate this as half and half, we only need to check the first and last K elements
    for i in range(DIST_BETWEEN_COPY):
        if order[0][-1] in order[1][:DIST_BETWEEN_COPY-i]:
            return False
    
    # each pair gets one occurence with border, one occurence without border
    #as a property of generate_random_shuffle, therefore we do not test for it.
    
    
    #TODO: We make sure that between adjacent pairs, the ending location and starting location is always opposite.
    #TODO: Also, make sure these distances obey MIN_DIST_PATH

    return True
        
def generate_valid_shuffle():
    shuffle = generate_random_shuffling()
    while not is_valid_shuffle(shuffle):
        shuffle = generate_random_shuffling()
    print("ee")
    return shuffle

In [26]:
shuffles = [generate_valid_shuffle() for i in range(NUM_SHUFFLES)]
shuffles

KeyboardInterrupt: 

## Checks

- We make sure that apmprox. half the pairs happen in A->B, and the other half happens in B->A
- We want to check the number of objects within the same side across a sliding window of 5 trials.

# Objects
- 5 different shapes (circle, cube, cylinder, pyramid, star)
- 5 different colors from a color-blind-friendly palette.
-> 25 different combinations.

Any object generated must not have been in the last 10 shown object.